# First Lookup on DemoParse Library


In [1]:
from demoparser2 import DemoParser
import pandas as pd
from pprint import pprint
from typing import Dict, Sequence, Optional, List, Tuple
import numpy as np

In [2]:
base_path = "../../demos"
parser = DemoParser(f"{base_path}/faceit/anubisFaceit.dem")

event_names = parser.list_game_events()

# Currently the event "all" gives you all events. Cursed solution for now
event_names

['item_pickup',
 'server_message',
 'inferno_startburn',
 'round_poststart',
 'hltv_chase',
 'round_announce_match_start',
 'bomb_pickup',
 'flashbang_detonate',
 'begin_new_match',
 'round_announce_warmup',
 'player_spawn',
 'smokegrenade_expired',
 'inferno_expire',
 'bomb_exploded',
 'player_death',
 'player_connect_full',
 'announce_phase_end',
 'round_announce_final',
 'round_time_warning',
 'round_prestart',
 'bomb_defused',
 'smokegrenade_detonate',
 'player_blind',
 'player_connect',
 'hegrenade_detonate',
 'weapon_fire',
 'weapon_reload',
 'player_team',
 'server_cvar',
 'hltv_fixed',
 'cs_pre_restart',
 'player_jump',
 'player_footstep',
 'item_equip',
 'player_hurt',
 'bomb_dropped',
 'show_survival_respawn_status',
 'bomb_planted',
 'decoy_started',
 'bomb_beginplant',
 'decoy_detonate',
 'chat_message',
 'bomb_begindefuse',
 'cs_win_panel_match',
 'weapon_zoom',
 'cs_round_start_beep',
 'player_disconnect',
 'round_announce_last_round_half',
 'cs_round_final_beep',
 'round

# Game Score

Halftime or final score


In [3]:
def get_player_average_damage_per_round():
    """
    Function to calculate the Average Damage per Round (ADR) for all players.

    Args:
        parser (DemoParser): An instance of DemoParser initialized with a demo file path.

    Returns:
        pd.DataFrame: A dataframe containing player Steam IDs and their ADR.
    """
    # Define the fields to extract
    wanted_fields = ["damage_total", "kills_total", "deaths_total"]

    # Parse the number of rounds
    round_end_df = parser.parse_event("round_end")
    return len(round_end_df) - 1  # Count the total number of rounds

    # # Parse the maximum tick to analyze final stats
    # max_tick = round_end_df["tick"].max()

    # # Parse the wanted fields for all players at the last tick
    # stats_df = parser.parse_ticks(wanted_fields, ticks=[max_tick])

    # # Ensure no NaN values in the stats
    # stats_df.fillna(0, inplace=True)

In [5]:
def get_final_score(
    players: Sequence[str] = None,
    round_info: str | int = "final",
) -> Tuple[pd.DataFrame, pd.DataFrame] | pd.DataFrame:
    """
    Retrieve the final score details for specified players or all players at the end of a round.

    Parameters:
    -----------
    players : Sequence[str], optional
        A list of player names to filter the DataFrame. If provided, the returned DataFrame 
        will only include data for the specified players. Defaults to None.

    round_info : str | int, optional
        Specifies the round for which to retrieve score data. Can take the following values:
        - "final" (default): Retrieves the final round score.
        - "half_time": Retrieves the score at halftime (end of round 12).
        - An integer: Retrieves the score for the specified round (e.g., 1 for the first round).
        If the integer is greater than the maximum round available, a ValueError is raised.

    Returns:
    --------
    Union[Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame], pd.DataFrame]
        - If `players` is provided: A single filtered DataFrame containing data for the specified players.
        - Otherwise: A tuple containing:
            1. DataFrame for Team 1
            2. DataFrame for Team 2
            3. DataFrame containing data for all players, sorted by team and KD ratio.

    Raises:
    -------
    ValueError
        - If `round_info` is not "final", "half_time", or a valid integer.
        - If `round_info` is an integer greater than the maximum round available.
    """

    
     # Determine the tick based on round_info
    events = parser.parse_event("round_end")
    max_round = len(events) - 1
    if isinstance(round_info, int):
        if round_info > max_round:
            raise ValueError(f"Invalid `round_info`: {round_info}. Maximum round is {max_round}.")
        tick = events.query(f"round == {round_info + 2}")["tick"].max()
    elif round_info == "half_time":
        tick = events.query("round == 14")["tick"].max()
    elif round_info == "final":
        tick = events["tick"].max()
    else:
        raise ValueError("Invalid `round_info` value. Must be 'final', 'half_time', or an integer.")

    wanted_fields = [
        "kills_total",
        "deaths_total",
        "mvps",
        "headshot_kills_total",
        "ace_rounds_total",
        "4k_rounds_total",
        "3k_rounds_total",
        "team_num",
        "damage_total",
        "assists_total",
        "team_score_first_half",
        "team_score_second_half",
    ]

    # Parse the ticks at the max_tick
    df = parser.parse_ticks(wanted_fields, ticks=[tick])    
    df["deaths_total"] = df["deaths_total"].fillna(0)

    # Calculate KD
    df["kd"] = np.where(
        df["deaths_total"] != 0,  
        round(df["kills_total"] / df["deaths_total"], 2),
        round((df["kills_total"] / 1), 2),
    )

    # Calculate HS %
    df["headshot_percentage"] = np.where(
    df["kills_total"] != 0,  
    round(df["headshot_kills_total"] / df["kills_total"] * 100),
    round((df["headshot_kills_total"] / 1) * 100),
    ).astype(int)   

    # Calculate ADR
    first_half_rounds = df["team_score_first_half"].unique().sum()
    second_half_rounds = df["team_score_second_half"].unique().sum()
    actual_rounds = first_half_rounds + second_half_rounds
    
    
    print(actual_rounds)
    if actual_rounds == 0:
        df["adr"] = round(df["damage_total"] / 1).astype(int)
    else:
        df["adr"] = round(df["damage_total"] / actual_rounds, 2)
    df["round"] = actual_rounds
    
    df["diff"] = df["kills_total"] - df["deaths_total"]
    
    df.sort_values("kd", inplace=True, ascending=False)
    
    # If specific players are provided, filter the DataFrame and return
    if players:
        df = df[df["name"].isin(players)]
        return df

    # Get unique team numbers dynamically
    unique_teams = df["team_num"].unique()

    # Assign the team numbers dynamically
    team_num_1, team_num_2 = unique_teams[:2]

    # Create separate DataFrames for each team
    df_team_1: pd.DataFrame = df[df["team_num"] == team_num_1].copy()
    df_team_2: pd.DataFrame = df[df["team_num"] == team_num_2].copy()

    return df_team_1, df_team_2, df.sort_values("team_num")


df = get_final_score(round_info=12)
df[2]

12


,kills_total,deaths_total,assists_total,headshot_kills_total,damage_total,ace_rounds_total,4k_rounds_total,3k_rounds_total,mvps,team_num,team_score_first_half,team_score_second_half,tick,steamid,name,kd,headshot_percentage,adr,round,diff
0,13,4,2,5,975,0,0,2,0,2,8,0,87273,76561198803359038,EvUsFDL,3.25,38,81.25,12,9
6,16,9,2,9,1412,0,0,2,4,2,8,0,87273,76561199075107764,PuliNFPS,1.78,56,117.67,12,7
3,10,6,3,7,923,0,0,1,1,2,8,0,87273,76561198077572982,cebola1_,1.67,70,76.92,12,4
4,8,6,5,2,1027,0,0,0,2,2,8,0,87273,76561198317791204,MIRAcls_,1.33,25,85.58,12,2
7,5,8,5,1,721,0,0,0,1,2,8,0,87273,76561198437803219,creepyfps,0.62,20,60.08,12,-3
8,10,10,1,7,1059,0,0,1,1,3,4,0,87273,76561198057428801,snowwJL,1.00,70,88.25,12,0
1,9,10,2,5,899,0,0,1,1,3,4,0,87273,76561198840306307,ton001,0.90,56,74.92,12,-1
2,7,11,1,3,760,0,0,0,2,3,4,0,87273,76561197999127404,vap999,0.64,43,63.33,12,-4
5,5,11,1,3,432,0,0,0,0,3,4,0,87273,76561197987161730,sr_jarbolas,0.45,60,36.00,12,-6
9,2,11,2,2,400,0,0,0,0,3,4,0,87273,76561198129776170,brbzinN,0.18,100,33.33,12,-9
